## Assistente de Tecnologia Especializado

In [ ]:
import openai

In [2]:
client = openai.Client()

In [ ]:
assistant = client.beta.assistants.create(
    name = "Tutor de Tecnologia",
    instructions = "Você é um tutor relacionado a assuntos de tecnologia",
    tools = [{"type":"code_interpreter"}],
    model="gpt-3.5-turbo-0125")

In [4]:
pergunta = """
Em um sistema de cache com mapeamento direto, dado uma memória principal de uma cache de 256 KB e bloco de 64 bytes:
- Quantas linhas existem na cache ?
- Onde o bloco de endereço 0x1A23C será mapeado na cache ?
"""

In [5]:
# Criação da Thread
thread = client.beta.threads.create()
message = client.beta.threads.messages.create(
    thread_id = thread.id,
    role = "user",
    content = pergunta)

C:\Users\Naja\AppData\Local\Temp\ipykernel_19436\3798620811.py:2: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  thread = client.beta.threads.create()
C:\Users\Naja\AppData\Local\Temp\ipykernel_19436\3798620811.py:3: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  message = client.beta.threads.messages.create(


In [7]:
# Executa a thread
run = client.beta.threads.runs.create(
    thread_id = thread.id,
    assistant_id = assistant.id,
    instructions = "Nome de usuário premium"
    )

C:\Users\Naja\AppData\Local\Temp\ipykernel_19436\3335153388.py:2: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  run = client.beta.threads.runs.create(


In [ ]:
# Exibe o status da thread
run.status

'queued'

In [9]:
# Aguarda o thread rodar
import time 
while run.status in ["queued","in_progress","cancelling"]:
    time.sleep(1)
    run = client.beta.threads.runs.retrieve(
        thread_id = thread.id,
        run_id = run.id
    )

C:\Users\Naja\AppData\Local\Temp\ipykernel_19436\3683720315.py:5: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  run = client.beta.threads.runs.retrieve(


In [12]:
# Verificar a resposta
if run.status == "completed":
    mensagens = client.beta.threads.messages.list(
        thread_id = thread.id
    )

    print(mensagens)
else:
    print(f"Erro: {run.status}")

C:\Users\Naja\AppData\Local\Temp\ipykernel_19436\1930001157.py:3: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  mensagens = client.beta.threads.messages.list(


SyncCursorPage[Message](data=[Message(id='msg_FkMiXfq7BSAQiZ3lHAc8r5yE', assistant_id='asst_LraJRBV5QeojCQdgSFa1v3cV', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='O bloco de endereço 0x1A23C será mapeado na linha 1672 da cache.'), type='text')], created_at=1756026401, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='assistant', run_id='run_OAxSj69ZcAGHpottjEk6HHlC', status=None, thread_id='thread_SMkTLrC9mSHV9vZ0NprkJKMr'), Message(id='msg_WHhzfKJePXCNzG3iA5C8Gp1P', assistant_id='asst_LraJRBV5QeojCQdgSFa1v3cV', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value="O endereço é dividido da seguinte forma:\n- 14 bits para a tag\n- 12 bits para o índice\n- Os bits da tag são: '00000000000000'\n- Os bits do índice são: '011010001000'\n\nAgora vamos calcular onde o bloco de endereço 0x1A23C será mapeado na cache."), type='text')], created_at=1756026399, incomplet

In [13]:
print(mensagens.data[0].content[0].text.value)

O bloco de endereço 0x1A23C será mapeado na linha 1672 da cache.


In [14]:
# Analisando os passos do modelo
run_steps = client.beta.threads.runs.steps.list(
    thread_id = thread.id,
    run_id = run.id)

C:\Users\Naja\AppData\Local\Temp\ipykernel_19436\764390232.py:2: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  run_steps = client.beta.threads.runs.steps.list(


In [ ]:
for step in run_steps.data[::-1]:
    print(f"\n===Step {step.step_details.type}")
    if step.step_details.type == "tool_calls":
        for tool_call in step.step_details.tool_calls:
            print("=" *10)
            print(tool_call.code_interpreter.input)
            print("=" *10)
    if step.step_details.type == "message_creation":
        message = client.beta.threads.messages.retrieve(
            thread_id = thread.id,
            message_id = step.step_details.message_creation.message_id)
        print(message.content[0].text.value)


===Step message_creation


C:\Users\Naja\AppData\Local\Temp\ipykernel_19436\977266563.py:9: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  message = client.beta.threads.messages.retrieve(


Para determinar quantas linhas existem na cache com mapeamento direto, precisamos calcular o número de linhas da cache. Esse valor pode ser obtido a partir da divisão do tamanho da cache pela tamanho do bloco.

Vamos realizar esses cálculos:

===Step tool_calls
# Tamanho da cache em bytes
tamanho_cache_kb = 256
tamanho_cache_bytes = tamanho_cache_kb * 1024

# Tamanho do bloco em bytes
tamanho_bloco_bytes = 64

# Calculando o número de linhas na cache
num_linhas_cache = tamanho_cache_bytes // tamanho_bloco_bytes

num_linhas_cache

===Step message_creation
A cache com mapeamento direto terá 4096 linhas.

Agora, para encontrar onde o bloco de endereço 0x1A23C será mapeado na cache, precisamos considerar a forma como o endereço é dividido em tag, índice e offset. No caso de um mapeamento direto, o índice é determinado pelo número de linhas na cache.

Vamos calcular onde o bloco de endereço 0x1A23C será mapeado na cache:

===Step tool_calls
import math

# Tamanho do endereço em bits
tamanho